In [12]:
import ml
from ml import Base

In [13]:
from typing import Any


class ML(ml.Base):
    def __init__(self):
        super().__init__()
        self.layer1 = ml.Linear(3, 2)
        self.layer2 = ml.Linear(2, 1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return x

In [14]:
ml = ML()

In [15]:
a = ml([1,2,3])
print(a)

[4.2298926669396915]


In [16]:
ml.parameters()

In [21]:
(1 * 0.44005809116875183 + 2 * 0.5987897077587271 + 3 * 0.9083886555355017)

4.362803473292711

In [22]:
(1 * 0.32934623871139923 + 2 * 0.11982311663789824 + 3 * 0.5321095629554001)

2.165321160853396

In [25]:
(0.3995027447994428 * 4.362803473292711 + 0.5305051645625966 * 2.165321160853396)

2.891666021370384